# MAT4110 implemented algorithms

In [5]:
import numpy as np

## Factorization and decomposition

### LU Decomposition

In [6]:
def lu_decomposition(mat):
    # Implementation of the LU decomposition algorithm for 
    # nxn matrix.
    L = np.zeros(mat.shape)
    U = np.zeros(mat.shape)
    A = mat.copy()
    
    for i in range(mat.shape[0]):
        # Calculate L, normalized
        L[:,i] = (1/A[:,i][i])*A[:,i]
        
        # Calculate U
        U[i,:] = A[i,:]
        
        # Calculate A_i
        A = A - np.outer(L[:,i],U[i,:])
        
    return L, U

In [7]:
mat = np.array([[3,4],[5,6]])
L, U = lu_decomposition(mat)
print("L\n", L, "\n")
print("U\n", U, "\n")
print(L.dot(U))


L
 [[ 1.         -0.        ]
 [ 1.66666667  1.        ]] 

U
 [[ 3.          4.        ]
 [ 0.         -0.66666667]] 

[[3. 4.]
 [5. 6.]]


### Cholesky factorization

In [8]:
def cholesky_factorization(A):                               
    """
    Performs cholesky factorization on a positive definite nxn
    matrix A.
    """   
    L = np.zeros(A.shape)
    D = np.zeros(A.shape)
    
    for i in range(A.shape[0]):
        # Set l_i, D_ii and calculate A_i
        L[:,i] = (1/A[i,i])*A[:,i]
        D[i,i] = A[i,i]
        A = A - D[i,i] * np.outer(L[:,i], L[:,i].T)
                                                             
    return L, D

In [9]:
mat = np.array([[3,4], [4,6]])
L, D = cholesky_factorization(mat)
print(L)
print(D)
print(L.T)
print(L.dot(D.dot(L.T)))

[[1.         0.        ]
 [1.33333333 1.        ]]
[[3.         0.        ]
 [0.         0.66666667]]
[[1.         1.33333333]
 [0.         1.        ]]
[[3. 4.]
 [4. 6.]]


### QR factorization (Gram-Schmidt)

In [10]:
def qr_factorization(A):
    """
    Factorize an n x m non-singular matrix A into two matrices
    Q and R such that A = QR, where n >= m
    This implementation uses the Gram-Schmidt algorithm.
    """
    Q = np.zeros(A.shape)
    R = np.zeros(A.shape)
    
    # Calculate the rest
    for i in range(A.shape[1]):
        w = A[:,i]
        
        for j in range(i):
            w = w - Q[:,j].dot(A[:,i])*Q[:,j]
        
        for j in range(i):
            R[j,i] = Q[:,j].dot(A[:,i])
        
        Q[:,i] = w/np.sqrt(w.dot(w))
        R[i,i] = np.sqrt(w.dot(w))
    
    return Q, R
        
        
    

In [11]:
mat = np.array([[2, 1, -3], [0, 0, -1], [0, 1, 4]])
Q, R = qr_factorization(mat)
print("Q =\n", Q)
print("R =\n", R)
print("QR =\n", Q.dot(R))

Q =
 [[ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]]
R =
 [[ 2.  1. -3.]
 [ 0.  1.  4.]
 [ 0.  0.  1.]]
QR =
 [[ 2.  1. -3.]
 [ 0.  0. -1.]
 [ 0.  1.  4.]]


### SVD - Singular Value Decomposition

In [110]:
def svd(A):
    """
    Compute the singular value decomposition of an nxm matrix A,
    such that A = USV^T where S contains the singular values.
    Using the equations
    A^T A = VS^TSV^T and
    AV = US
    """
    n, m = A.shape
    U = np.zeros((n,n))
    S = np.zeros((n,m))
    V = np.zeros((m,m))
    
    # Calculate eigenvalues and eigenvectors
    w, v = np.linalg.eig(A.T.dot(A))
    
    # Sort in descending order
    indices = np.argsort(-w)
    w = -np.sort(-w)
    for i in range(len(w)):
        V[:,i] = v[:,indices[i]]
    for i in range(len(w)):
        S[i,i] = np.sqrt(w[i])
    
    
    A_V = A.dot(V)
    
    for i in range(n):
        U[:,i] = A_V[:,i]/S[i,i]
    
    return U, S, V
    
    

In [111]:
mat = np.array([[5, 5], [-1, 7]])
U, S, V = svd(mat)
A = U.dot(S).dot(V.T)
print(U)
print(S)
print(V.T)
print("")
svd = np.linalg.svd(mat)
print(svd[0])
print(svd[1])
print(svd[2])

V
[[-0.31622777 -0.9486833 ]
 [-0.9486833   0.31622777]]
[[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]
[[8.94427191 0.        ]
 [0.         4.47213595]]
[[-0.31622777 -0.9486833 ]
 [-0.9486833   0.31622777]]

[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
[8.94427191 4.47213595]
[[ 0.31622777  0.9486833 ]
 [ 0.9486833  -0.31622777]]


## Solving linear systems of equations

### Forward substitution

In [14]:
def forward_substitution(mat_A, b):                          
    """                                                      
    Apply forward substitution to a lower triangular matrix A
    and return the vector x in the matrix equation Ax = b    
    """                                                      
    N = len(mat_A[0])                                        
    x = np.zeros(N)                                          
    for i in range(N):                                       
        sum = 0                                              
        for j in range(i):                                   
            sum += mat_A[i,j]*x[j]                           
        x[i] = (b[i] - sum)/mat_A[i,i]                       
    return x                                                 


### Back substitution

In [15]:
def back_substitution(mat, b):                           
    """                                                    
    Apply back substitution to an upper triangular matrix A
    and return the vector x in the matrix equation Ax = b  
    """                                                    
    N = len(mat[0])                                      
    x = np.zeros(N)                                        
    for i in reversed(range(N)):                           
        sum = 0                                            
        for j in range(i+1,N):                             
            sum += mat[i,j]*x[j]                         
        x[i] = (b[i] - sum)/mat[i,i]                     
    return x                                               
